In [ ]:
%load_ext autoreload
%autoreload 2
from models import *
from fetch_data import *

DATASET

In [2]:
x_train, y_train,x_test, y_test=generate_data(name_data='mnist')

In [3]:
# plt.imshow(x_train[7])

HYPERPARAMETERS

In [ ]:
spectral_config = {  'is_base_trainable': True,
                     'is_diag_start_trainable': False,
                     'is_diag_end_trainable': False,
                     'use_bias': True
                    }

spectral_cnn1d_config ={ 'kernel_size': 3,
                         'stride': 2,
                         'padding': 0,
                         'trainable_phi':False,
                         'use_lambda_out':False,
                         'use_lambda_in' : True,
                         'use_bias': True}

spectral_cnn2d_config={ 'kernel_size': 3,
                        'strides': 1,
                        'padding': 'VALID',
                        "use_lambda_out":False,
                        "use_lambda_in":False,
                        "use_encode":False,
                        "use_decode":False,
                        'use_bias': False,
                        "trainable_omega_diag":True,
                        "trainable_omega_triu":True,
                        "trainable_omega_tril":True,
                        "trainable_aggregate":True,
                        "kernel_initializer":"glorot_uniform"
                        }

maxpooling_config ={ 'pool_size': (2,2),
                     'strides': 2,
                     'padding': 'valid' }

hyperparameters = { 'filters' :2,
                    'input_shape' :(28,28,1),
                    'learning_rate' : 0.01,
                    'epochs' : 20,
                    'batch_size' :100,
                    'activation': 'relu',
                    'labels' : 10}





In [12]:
models = SpectralCnn(spectral_config=spectral_config,
                     spectral_cnn1d_config=spectral_cnn1d_config,
                     spectral_cnn2d_config=spectral_cnn2d_config,
                     hyperparameters=hyperparameters,
                     maxpooling_config=maxpooling_config)

In [13]:
print("build model")
models.compile_models(units=1000, Use_pruning=False,pruning=0.1, use_base_and_Lambda=True, name='reference')
models.summary(name='reference')


build model
Reference model summary:
 


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ spec_cnn2d_2 (SpecCnn2D)        │ (None, 26, 26, 2)      │            20 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 13, 13, 2)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 338)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1000)           │       339,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │        10,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 349,030 (1.33 MB)

 Trainable params: 349,030 (1.33 MB)

 Non-trainable params: 0 (0.00 B)

In [43]:
models.mask_by_quantile([1,24,3,-1,25,6,8],0.0)

<tf.Tensor: shape=(7,), dtype=int32, numpy=array([ 1, 24,  3, -1, 25,  6,  8], dtype=int32)>

In [14]:
models.train(x_train, y_train, x_test, y_test, name='reference',verbose=1,layers=(3,4))

Epoch 1/20
600/600 ━━━━━━━━━━━━━━━━━━━━ 12s 18ms/step - accuracy: 0.7190 - loss: 0.8073 - val_accuracy: 0.9593 - val_loss: 0.1279
Epoch 2/20
600/600 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.9736 - loss: 0.0848 - val_accuracy: 0.9772 - val_loss: 0.0715
Epoch 3/20
600/600 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.9811 - loss: 0.0596 - val_accuracy: 0.9726 - val_loss: 0.0889
Epoch 4/20
600/600 ━━━━━━━━━━━━━━━━━━━━ 11s 19ms/step - accuracy: 0.9851 - loss: 0.0436 - val_accuracy: 0.9805 - val_loss: 0.0632
Epoch 5/20
600/600 ━━━━━━━━━━━━━━━━━━━━ 11s 19ms/step - accuracy: 0.9873 - loss: 0.0387 - val_accuracy: 0.9803 - val_loss: 0.0670
Epoch 6/20
600/600 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.9904 - loss: 0.0293 - val_accuracy: 0.9828 - val_loss: 0.0643
Epoch 7/20
600/600 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.9925 - loss: 0.0212 - val_accuracy: 0.9820 - val_loss: 0.0647
Epoch 8/20
600/600 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.9923 - loss: 0.0225 - 

In [56]:
models.evaluate(x_test, y_test,order=0 , name='reference',layers=(3,4),p=0.5)

100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5029 - loss: 1.6511
Accuracy of reference model: 0.511900007724762


In [40]:
name="reference"
models_name=['reference']
replication = {'rep':10}  # Number of replications
#depth={'N':[50,100, 150, 200, 500, 800, 1000, 1500, 2000, 2500]}
depth={'N':[2000]}
drop={'p':[0.1,0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]}
accuracy:Dict[int,Dict[int,pd.DataFrame]]={i: {} for i in range(10)}
for k,p in enumerate(drop.get('p')):
    for order in range(replication['rep']):
        with open(f"Robustness/Data/{name}/accuracy{p}_{order}.txt", "r") as f:
            accuracy[k][order]=pd.read_csv(f, sep=" ", header=None)
pd.DataFrame({p: [value.loc[0,0] for value in accuracy[k].values()] for k,p in  enumerate(drop.get('p'))}).T


,0,1,2,3,4,5,6,7,8,9
0.1,0.8785,0.8760,0.8605,0.8725,0.8600,0.8615,0.8495,0.8375,0.8530,0.8180
0.2,0.7870,0.7690,0.7720,0.7210,0.8040,0.7730,0.7825,0.7240,0.7325,0.7930
0.3,0.6790,0.6850,0.6315,0.6080,0.6295,0.6590,0.6250,0.6780,0.7105,0.6985
0.4,0.5780,0.5795,0.5530,0.4790,0.6265,0.5920,0.5015,0.5255,0.5825,0.5700
0.5,0.4460,0.4860,0.4185,0.3680,0.3785,0.4510,0.4810,0.4715,0.4840,0.4010
0.6,0.3470,0.3505,0.2900,0.3875,0.3775,0.3620,0.2810,0.2895,0.3675,0.3025
0.7,0.2750,0.2325,0.2700,0.2550,0.1810,0.2810,0.2290,0.2875,0.2945,0.2075
0.8,0.1440,0.2215,0.1435,0.2140,0.1130,0.2200,0.2385,0.1585,0.1750,0.1845
0.9,0.1025,0.1135,0.1055,0.1235,0.1130,0.1375,0.1100,0.1055,0.1050,0.1060
1.0,0.1055,0.1055,0.1055,0.1055,0.0985,0.1055,0.0985,0.1055,0.0985,0.1055


In [ ]:
accuracy[0].values())

,0
0,0.774


In [39]:
pd.DataFrame({p: [value.loc[0,0] for value in accuracy[k].values()] for k,p in  enumerate(drop.get('p'))}).T

,0,1,2,3,4,5,6,7,8,9
0.1,0.7740,0.7950,0.5755,0.6805,0.6480,0.7405,0.6230,0.6925,0.7895,0.7610
0.2,0.7170,0.7255,0.5035,0.6255,0.5805,0.7155,0.5415,0.6540,0.7375,0.7200
0.3,0.6980,0.6565,0.4615,0.4670,0.4640,0.6875,0.5250,0.6460,0.7250,0.7200
0.4,0.6975,0.5670,0.4420,0.4330,0.4640,0.6685,0.5160,0.6445,0.7215,0.7220
0.5,0.6965,0.5085,0.4345,0.3815,0.4675,0.6680,0.5155,0.6435,0.7130,0.7200
0.6,0.6925,0.5050,0.4360,0.3800,0.4670,0.6610,0.5070,0.6430,0.7110,0.7095
0.7,0.6515,0.5030,0.4175,0.3790,0.4660,0.6375,0.5140,0.6380,0.7095,0.7015
0.8,0.6195,0.4990,0.4075,0.3795,0.4330,0.4920,0.5115,0.6325,0.6820,0.6265
0.9,0.5690,0.4690,0.3650,0.3425,0.3825,0.4730,0.4810,0.6075,0.6725,0.3450
1.0,0.3700,0.2850,0.1960,0.2665,0.2075,0.2825,0.2940,0.3065,0.2440,0.2215


# OTHERS ANALYSIS 

In [ ]:
# Exemple de liste de réels
# valeurs = [1.2, 2.5, 1.2, 3.8, 2.5, 2.5, 4.0, 3.8, 1.2]
# Créer un histogramme
plt.hist(value[0].numpy().tolist(), bins='auto', edgecolor='black', density=True)
# Ajouter des titres et labels
plt.xlabel(r"$\Lambda^{in}$")
plt.ylabel("freq")
plt.title(r"Repartition des valeurs de $\Lambda^{in}$")
plt.grid(True)
plt.savefig('Lambda.pdf')
plt.show()

In [ ]:
spectral_config = {  'is_base_trainable': False,
                     'is_diag_start_trainable': False,
                     'is_diag_end_trainable': True,
                     'use_bias': True
                    }

spectral_cnn1d_config ={ 'kernel_size': 3,
                         'stride': 2,
                         'padding': 0,
                         'trainable_phi':False,
                         'use_lambda_out':False,
                         'use_lambda_in' : True,
                         'use_bias': True}

maxpooling_config ={ 'pool_size': 2,
                     'strides': 1,
                     'padding': 'valid' }

hyperparameters = { 'filters' :20,
                    'input_shape' :40,
                    'learning_rate' : 0.01,
                    'epochs' : 20,
                    'batch_size' :100,
                    'activation': 'relu',
                    'labels' : 10}

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=(hyperparameters.get('input_shape'),)))

#model.add(Conv1D(filters=20, kernel_size=3, strides=1,activation='relu', padding='valid'))

model.add(SpecCnn1d_(filters=hyperparameters.get('filters'), **spectral_cnn1d_config,activation=hyperparameters.get('activation')))

model.add(tf.keras.layers.MaxPooling1D(**maxpooling_config))

model.add(tf.keras.layers.Flatten())

model.add(Spectral(2000, **spectral_config, activation=hyperparameters.get('activation')))
model.add(Spectral(hyperparameters.get('labels'), **spectral_config, activation='softmax'))

opt = tf.keras.optimizers.Adam(learning_rate=hyperparameters.get('learning_rate'))
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(x_train, y_train, epochs=hyperparameters.get('epochs'), batch_size=hyperparameters.get('batch_size'), validation_data=(x_test, y_test))